In [ ]:
import os
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime as datet
import json
from multiprocessing import Pool
from brian2 import *

from model import IzhikevichNetwork, plot_raster_results
from metrics import plot_connectivity_dashboard, analyze_simulation_results, plot_population_dashboard, print_network_statistics_table

def create_summary_table(connectivity_results, network_stats):
    """Crear tabla resumen con métricas clave - FIXED VERSION"""
    
    summary_data = []
    
    for condition, conn_res in connectivity_results.items():
        if conn_res is None:
            continue
        
        # Parse condition name to extract noise_type and noise_level
        # Expected format: "noise_type_noise_level" (e.g., "gaussian_4", "poisson_6")
        parts = condition.split('_')
        if len(parts) >= 2:
            noise_type = parts[0]
            noise_level = int(parts[1])
        else:
            # Fallback: try to infer from condition name
            if 'gaussian' in condition.lower():
                noise_type = 'gaussian'
            elif 'poisson' in condition.lower():
                noise_type = 'poisson'
            else:
                noise_type = 'unknown'
            
            # Try to extract number from condition
            import re
            numbers = re.findall(r'\d+', condition)
            noise_level = int(numbers[0]) if numbers else 0
        
        net_stats = network_stats[condition]
        
        row = {
            'condition': condition,
            'noise_type': noise_type,  # ADD THIS LINE
            'noise_level': noise_level,
            'cross_corr_peak': abs(conn_res['cross_correlation']['peak_value']),
            'cross_corr_lag_ms': conn_res['cross_correlation']['peak_lag'],
            'plv_alpha': conn_res['plv_pli']['alpha']['plv'],
            'pli_alpha': conn_res['plv_pli']['alpha']['pli'],
            'plv_gamma': conn_res['plv_pli']['gamma']['plv'],
            'pli_gamma': conn_res['plv_pli']['gamma']['pli'],
            'coherence_peak': conn_res['coherence']['peak_coherence'],
            'coherence_freq_hz': conn_res['coherence']['peak_freq'],
            'alpha_coherence': conn_res['coherence']['alpha_coherence'],
            'gamma_coherence': conn_res['coherence']['gamma_coherence'],
            'timescale_A_ms': conn_res['int_A']['tau'],
            'timescale_B_ms': conn_res['int_B']['tau'],
            'fit_quality_A': conn_res['int_A']['fit_quality'],
            'fit_quality_B': conn_res['int_B']['fit_quality'],
            'firing_rate_exc_A': net_stats['A']['freq_exc'],
            'firing_rate_exc_B': net_stats['B']['freq_exc'],
            'cv_A': net_stats['A']['cv'],
            'cv_B': net_stats['B']['cv'],
            'sync_level_A': net_stats['A']['sync_level'],
            'sync_level_B': net_stats['B']['sync_level'],
            'burst_rate_A': net_stats['A']['burst_rate'],
            'burst_rate_B': net_stats['B']['burst_rate']
        }
        
        summary_data.append(row)
    
    return summary_data

def print_experiment_summary(connectivity_results, summary_df, results_dir, delay, noise_levels):
    """Imprimir resumen comprehensivo del experimento"""
    
    print(f"\n{'='*80}")
    print(f"RESUMEN FINAL - EXPERIMENTO 5: RÉGIMEN DE ACTIVIDAD")
    print(f"{'='*80}")
    
    print(f"📋 CONFIGURACIÓN:")
    print(f"   • Delay fijo: {delay}ms")
    print(f"   • Noise levels: {noise_levels}")
    print(f"   • dt: 0.01ms (alta resolución)")
    print(f"   • Duración: 1200ms (warmup: 200ms)")
    print(f"   • Ejecución: PARALELA")
    
    print(f"\n🧠 REGÍMENES IDENTIFICADOS:")
    for _, row in summary_df.iterrows():
        noise = int(row['noise_level'])
        freq_A = row['firing_rate_exc_A']
        freq_B = row['firing_rate_exc_B']
        sync = row['sync_level_A']
        cross_corr = row['cross_corr_peak']
        lag = row['cross_corr_lag_ms']
        
        if freq_A < 0.1:
            regime = "SILENCIOSO"
        elif freq_A < 1.0:
            regime = "ACTIVO BAJO"
        elif freq_A < 3.0:
            regime = "ACTIVO"
        else:
            regime = "BURSTING"
        
        print(f"   • Noise {noise:2d}: {regime:12s} | Freq: {freq_A:5.2f}Hz | Sync: {sync:10s} | Cross-corr: {cross_corr:5.3f} (lag {lag:4.0f}ms)")
    
    print(f"\n🔗 EFECTOS DEL DELAY (5ms):")
    observable_delays = summary_df[abs(summary_df['cross_corr_lag_ms']) > 10]
    if len(observable_delays) > 0:
        print(f"   • Delay observable en noise levels: {list(observable_delays['noise_level'].astype(int))}")
        for _, row in observable_delays.iterrows():
            print(f"     - Noise {int(row['noise_level'])}: lag {row['cross_corr_lag_ms']:.0f}ms (amplificación x{abs(row['cross_corr_lag_ms'])/5:.1f})")
    else:
        print(f"   • Delay NO observable en ningún régimen")
    
    print(f"\n📊 SINCRONIZACIÓN:")
    best_sync = summary_df.loc[summary_df['cross_corr_peak'].idxmax()]
    worst_sync = summary_df.loc[summary_df['cross_corr_peak'].idxmin()]
    print(f"   • Mayor sincronización: Noise {int(best_sync['noise_level'])} (cross-corr: {best_sync['cross_corr_peak']:.3f})")
    print(f"   • Menor sincronización: Noise {int(worst_sync['noise_level'])} (cross-corr: {worst_sync['cross_corr_peak']:.3f})")
    
    # PLV analysis
    alpha_plv_mean = summary_df['plv_alpha'].mean()
    gamma_plv_mean = summary_df['plv_gamma'].mean()
    print(f"   • PLV promedio Alpha: {alpha_plv_mean:.3f} | Gamma: {gamma_plv_mean:.3f}")
    
    print(f"\n⚡ ACTIVIDAD POBLACIONAL:")
    print(f"   • Rango frecuencias Pop A: {summary_df['firing_rate_exc_A'].min():.3f} - {summary_df['firing_rate_exc_A'].max():.3f} Hz")
    print(f"   • Rango frecuencias Pop B: {summary_df['firing_rate_exc_B'].min():.3f} - {summary_df['firing_rate_exc_B'].max():.3f} Hz")
    print(f"   • CV rango: {summary_df['cv_A'].min():.3f} - {summary_df['cv_A'].max():.3f}")
    
    print(f"\n🎯 CONCLUSIONES CLAVE:")
    print(f"   • Delay de 5ms solo observable en régimen intermedio")
    print(f"   • Regímenes extremos enmascaran efectos del delay")
    print(f"   • Sincronización máxima en régimen silencioso")
    print(f"   • Transición muestra cambio dramático en timing")
    
    # Quality checks
    good_fits = summary_df[(summary_df['fit_quality_A'] == 'good') & (summary_df['fit_quality_B'] == 'good')]
    print(f"\n🔧 CALIDAD ANÁLISIS:")
    print(f"   • Fits exponenciales buenos: {len(good_fits)}/{len(summary_df)} condiciones")
    print(f"   • Coherencia espectral promedio: {summary_df['coherence_peak'].mean():.3f}")
    
    print(f"\n📁 ARCHIVOS GENERADOS:")
    print(f"   • Directorio: {results_dir}")
    print(f"   • Dashboards: connectivity_dashboard.png")
    print(f"   • Datos: summary_metrics.csv, trends_summary.png")
    print(f"   • Rasters: raster_noise_*.png")
    
    print(f"\n{'='*80}")

def plot_summary_trends(summary_df, results_dir):
    """Enhanced trends plotting with noise type separation and additional metrics"""
    
    # Separate data by noise type - FIXED VERSION
    # First, let's see what conditions we actually have
    print(f"Available conditions: {summary_df['noise_level'].unique()}")
    print(f"Summary DataFrame shape: {summary_df.shape}")
    print(f"Summary DataFrame columns: {summary_df.columns.tolist()}")
    
    # Check if we have a 'noise_type' column, if not, infer from row order
    if 'noise_type' not in summary_df.columns:
        # Infer noise type from row order (assuming gaussian first, then poisson)
        n_rows = len(summary_df)
        n_conditions_per_type = n_rows // 2
        
        noise_types = ['gaussian'] * n_conditions_per_type + ['poisson'] * n_conditions_per_type
        summary_df = summary_df.copy()
        summary_df['noise_type'] = noise_types
    
    # Now separate by actual noise type
    gaussian_data = summary_df[summary_df['noise_type'] == 'gaussian']
    poisson_data = summary_df[summary_df['noise_type'] == 'poisson']
    
    print(f"Gaussian data shape: {gaussian_data.shape}")
    print(f"Poisson data shape: {poisson_data.shape}")
    
    # Check if we have data for both types
    if len(gaussian_data) == 0:
        print("Warning: No Gaussian data found")
        return None
    if len(poisson_data) == 0:
        print("Warning: No Poisson data found")
        return None
    
    fig, axes = plt.subplots(3, 3, figsize=(18, 15))
    
    # Improved styling
    gaussian_style = {'color': '#2E86AB', 'marker': 'o', 'linewidth': 3, 'markersize': 10, 
                      'markeredgecolor': 'white', 'markeredgewidth': 2}
    poisson_style = {'color': '#F24236', 'marker': '^', 'linewidth': 3, 'markersize': 10,
                     'markeredgecolor': 'white', 'markeredgewidth': 2}
    
    # 1. Cross-correlation peak vs noise
    axes[0,0].plot(gaussian_data['noise_level'], gaussian_data['cross_corr_peak'], 
                   label='Gaussian', **gaussian_style)
    axes[0,0].plot(poisson_data['noise_level'], poisson_data['cross_corr_peak'], 
                   label='Poisson', **poisson_style)
    axes[0,0].set_xlabel('Noise Level')
    axes[0,0].set_ylabel('Cross-correlation Peak')
    axes[0,0].set_title('Synchronization vs Noise Type')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)
    
    # 2. Cross-correlation lag vs noise
    axes[0,1].plot(gaussian_data['noise_level'], gaussian_data['cross_corr_lag_ms'], 'o-', 
                   label='Gaussian', color='blue', linewidth=2, markersize=8)
    axes[0,1].plot(poisson_data['noise_level'], poisson_data['cross_corr_lag_ms'], 's-', 
                   label='Poisson', color='red', linewidth=2, markersize=8)
    axes[0,1].set_xlabel('Noise Level')
    axes[0,1].set_ylabel('Cross-correlation Lag (ms)')
    axes[0,1].set_title('Temporal Delay vs Noise Type')
    axes[0,1].legend()
    axes[0,1].grid(True, alpha=0.3)
    axes[0,1].axhline(y=6, color='black', linestyle='--', alpha=0.5, label='Expected Delay')
    
    # 3. PLV Alpha vs noise
    axes[0,2].plot(gaussian_data['noise_level'], gaussian_data['plv_alpha'], 'o-', 
                   label='Gaussian PLV', color='blue', linewidth=2, markersize=8)
    axes[0,2].plot(poisson_data['noise_level'], poisson_data['plv_alpha'], 's-', 
                   label='Poisson PLV', color='red', linewidth=2, markersize=8)
    axes[0,2].plot(gaussian_data['noise_level'], gaussian_data['pli_alpha'], 'o--', 
                   label='Gaussian PLI', color='lightblue', linewidth=1.5, markersize=6)
    axes[0,2].plot(poisson_data['noise_level'], poisson_data['pli_alpha'], 's--', 
                   label='Poisson PLI', color='pink', linewidth=1.5, markersize=6)
    axes[0,2].set_xlabel('Noise Level')
    axes[0,2].set_ylabel('Phase Locking (Alpha)')
    axes[0,2].set_title('Alpha Band Phase Locking')
    axes[0,2].legend()
    axes[0,2].grid(True, alpha=0.3)
    
    # 4. Firing rates vs noise
    axes[1,0].plot(gaussian_data['noise_level'], gaussian_data['firing_rate_exc_A'], 'o-', 
                   label='Gaussian Pop A', color='blue', linewidth=2, markersize=8)
    axes[1,0].plot(gaussian_data['noise_level'], gaussian_data['firing_rate_exc_B'], 'o--', 
                   label='Gaussian Pop B', color='lightblue', linewidth=1.5, markersize=6)
    axes[1,0].plot(poisson_data['noise_level'], poisson_data['firing_rate_exc_A'], 's-', 
                   label='Poisson Pop A', color='red', linewidth=2, markersize=8)
    axes[1,0].plot(poisson_data['noise_level'], poisson_data['firing_rate_exc_B'], 's--', 
                   label='Poisson Pop B', color='pink', linewidth=1.5, markersize=6)
    axes[1,0].set_xlabel('Noise Level')
    axes[1,0].set_ylabel('Firing Rate (Hz)')
    axes[1,0].set_title('Population Activity vs Noise')
    axes[1,0].legend()
    axes[1,0].grid(True, alpha=0.3)
    
    # 5. CV vs noise
    axes[1,1].plot(gaussian_data['noise_level'], gaussian_data['cv_A'], 'o-', 
                   label='Gaussian Pop A', color='blue', linewidth=2, markersize=8)
    axes[1,1].plot(gaussian_data['noise_level'], gaussian_data['cv_B'], 'o--', 
                   label='Gaussian Pop B', color='lightblue', linewidth=1.5, markersize=6)
    axes[1,1].plot(poisson_data['noise_level'], poisson_data['cv_A'], 's-', 
                   label='Poisson Pop A', color='red', linewidth=2, markersize=8)
    axes[1,1].plot(poisson_data['noise_level'], poisson_data['cv_B'], 's--', 
                   label='Poisson Pop B', color='pink', linewidth=1.5, markersize=6)
    axes[1,1].set_xlabel('Noise Level')
    axes[1,1].set_ylabel('CV (Coefficient of Variation)')
    axes[1,1].set_title('Firing Irregularity vs Noise')
    axes[1,1].legend()
    axes[1,1].grid(True, alpha=0.3)
    
    # 6. Timescales vs noise (filter out NaN values)
    gaussian_valid = gaussian_data.dropna(subset=['timescale_A_ms', 'timescale_B_ms'])
    poisson_valid = poisson_data.dropna(subset=['timescale_A_ms', 'timescale_B_ms'])
    
    if len(gaussian_valid) > 0:
        axes[1,2].plot(gaussian_valid['noise_level'], gaussian_valid['timescale_A_ms'], 'o-', 
                       label='Gaussian Pop A', color='blue', linewidth=2, markersize=8)
        axes[1,2].plot(gaussian_valid['noise_level'], gaussian_valid['timescale_B_ms'], 'o--', 
                       label='Gaussian Pop B', color='lightblue', linewidth=1.5, markersize=6)
    if len(poisson_valid) > 0:
        axes[1,2].plot(poisson_valid['noise_level'], poisson_valid['timescale_A_ms'], 's-', 
                       label='Poisson Pop A', color='red', linewidth=2, markersize=8)
        axes[1,2].plot(poisson_valid['noise_level'], poisson_valid['timescale_B_ms'], 's--', 
                       label='Poisson Pop B', color='pink', linewidth=1.5, markersize=6)
    axes[1,2].set_xlabel('Noise Level')
    axes[1,2].set_ylabel('Intrinsic Timescale (ms)')
    axes[1,2].set_title('Memory Timescales vs Noise')
    axes[1,2].legend()
    axes[1,2].grid(True, alpha=0.3)
    
    # 7. Gamma band metrics
    axes[2,0].plot(gaussian_data['noise_level'], gaussian_data['plv_gamma'], 'o-', 
                   label='Gaussian PLV γ', color='blue', linewidth=2, markersize=8)
    axes[2,0].plot(poisson_data['noise_level'], poisson_data['plv_gamma'], 's-', 
                   label='Poisson PLV γ', color='red', linewidth=2, markersize=8)
    axes[2,0].plot(gaussian_data['noise_level'], gaussian_data['gamma_coherence'], 'o--', 
                   label='Gaussian Coh γ', color='lightblue', linewidth=1.5, markersize=6)
    axes[2,0].plot(poisson_data['noise_level'], poisson_data['gamma_coherence'], 's--', 
                   label='Poisson Coh γ', color='pink', linewidth=1.5, markersize=6)
    axes[2,0].set_xlabel('Noise Level')
    axes[2,0].set_ylabel('Gamma Band Metrics')
    axes[2,0].set_title('Gamma Synchronization vs Noise')
    axes[2,0].legend()
    axes[2,0].grid(True, alpha=0.3)
    
    # 8. Activity regime classification
    def classify_regime(firing_rate):
        if firing_rate < 0.1:
            return 0  # Silent
        elif firing_rate < 1.0:
            return 1  # Low activity
        elif firing_rate < 10.0:
            return 2  # Active
        else:
            return 3  # Bursting
    
    gaussian_regimes = [classify_regime(fr) for fr in gaussian_data['firing_rate_exc_A']]
    poisson_regimes = [classify_regime(fr) for fr in poisson_data['firing_rate_exc_A']]
    
    regime_names = ['Silent', 'Low Active', 'Active', 'Bursting']
    
    axes[2,1].scatter(gaussian_data['noise_level'], gaussian_regimes, 
                      s=150, c='blue', marker='o', label='Gaussian', alpha=0.7)
    axes[2,1].scatter(poisson_data['noise_level'], poisson_regimes, 
                      s=150, c='red', marker='s', label='Poisson', alpha=0.7)
    axes[2,1].set_xlabel('Noise Level')
    axes[2,1].set_ylabel('Activity Regime')
    axes[2,1].set_yticks(range(4))
    axes[2,1].set_yticklabels(regime_names)
    axes[2,1].set_title('Network Activity Regimes')
    axes[2,1].legend()
    axes[2,1].grid(True, alpha=0.3)
    
    # 9. Delay detection efficacy
    def delay_detection_score(cross_corr_peak, lag_ms, expected_delay=6):
        """Score how well the delay is detected: high correlation + correct lag = high score"""
        lag_accuracy = 1.0 - min(abs(lag_ms - expected_delay) / 20.0, 1.0)  # Penalty for wrong lag
        return cross_corr_peak * lag_accuracy
    
    gaussian_scores = [delay_detection_score(peak, lag) for peak, lag in 
                       zip(gaussian_data['cross_corr_peak'], gaussian_data['cross_corr_lag_ms'])]
    poisson_scores = [delay_detection_score(peak, lag) for peak, lag in 
                      zip(poisson_data['cross_corr_peak'], poisson_data['cross_corr_lag_ms'])]
    
    axes[2,2].plot(gaussian_data['noise_level'], gaussian_scores, 'o-', 
                   label='Gaussian', color='blue', linewidth=2, markersize=8)
    axes[2,2].plot(poisson_data['noise_level'], poisson_scores, 's-', 
                   label='Poisson', color='red', linewidth=2, markersize=8)
    axes[2,2].set_xlabel('Noise Level')
    axes[2,2].set_ylabel('Delay Detection Score')
    axes[2,2].set_title('Delay Detection Efficacy')
    axes[2,2].legend()
    axes[2,2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f"{results_dir}/enhanced_trends_analysis.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    # Generate summary statistics table - FIXED VERSION
    print("\n" + "="*80)
    print("ENHANCED ANALYSIS SUMMARY")
    print("="*80)
    
    print(f"\n📊 NOISE TYPE COMPARISON:")
    print(f"{'Metric':<25} {'Gaussian':<15} {'Poisson':<15} {'Difference':<15}")
    print("-" * 70)
    
    # Compare means across noise levels
    gaussian_mean_sync = gaussian_data['cross_corr_peak'].mean()
    poisson_mean_sync = poisson_data['cross_corr_peak'].mean()
    print(f"{'Synchronization':<25} {gaussian_mean_sync:.3f}{'':<10} {poisson_mean_sync:.3f}{'':<10} {abs(gaussian_mean_sync-poisson_mean_sync):.3f}")
    
    gaussian_mean_lag = gaussian_data['cross_corr_lag_ms'].mean()
    poisson_mean_lag = poisson_data['cross_corr_lag_ms'].mean()
    print(f"{'Mean Lag (ms)':<25} {gaussian_mean_lag:.1f}{'':<14} {poisson_mean_lag:.1f}{'':<14} {abs(gaussian_mean_lag-poisson_mean_lag):.1f}")
    
    gaussian_mean_freq = gaussian_data['firing_rate_exc_A'].mean()
    poisson_mean_freq = poisson_data['firing_rate_exc_A'].mean()
    print(f"{'Firing Rate (Hz)':<25} {gaussian_mean_freq:.2f}{'':<12} {poisson_mean_freq:.2f}{'':<12} {abs(gaussian_mean_freq-poisson_mean_freq):.2f}")
    
    print(f"\n🎯 DELAY DETECTION ANALYSIS:")
    # Find best delay detection for each noise type - FIXED VERSION
    if len(gaussian_scores) > 0:
        best_gaussian_idx = np.argmax(gaussian_scores)
        print(f"Best Gaussian detection: Noise {gaussian_data.iloc[best_gaussian_idx]['noise_level']} "
              f"(score: {gaussian_scores[best_gaussian_idx]:.3f}, lag: {gaussian_data.iloc[best_gaussian_idx]['cross_corr_lag_ms']:.0f}ms)")
    else:
        print("No Gaussian data available")
    
    if len(poisson_scores) > 0:
        best_poisson_idx = np.argmax(poisson_scores)
        print(f"Best Poisson detection:  Noise {poisson_data.iloc[best_poisson_idx]['noise_level']} "
              f"(score: {poisson_scores[best_poisson_idx]:.3f}, lag: {poisson_data.iloc[best_poisson_idx]['cross_corr_lag_ms']:.0f}ms)")
    else:
        print("No Poisson data available")
    
    print(f"\n🧠 REGIME TRANSITIONS:")
    # Ensure we have data to compare
    min_len = min(len(gaussian_data), len(poisson_data))
    for i in range(min_len):
        noise_level = gaussian_data.iloc[i]['noise_level']
        g_regime = gaussian_regimes[i] if i < len(gaussian_regimes) else 0
        p_regime = poisson_regimes[i] if i < len(poisson_regimes) else 0
        print(f"Noise {noise_level:2d}: {regime_names[g_regime]:12s} → {regime_names[p_regime]:12s}")
        
    return fig

In [ ]:
# Create analyzer and run analysis

from metrics import NeuralConnectivityAnalyzer
analyzer = NeuralConnectivityAnalyzer(analysis_dt=0.01*ms)  # Use actual dt

def run_single_noise_simulation(args):
    """Worker function for single noise level simulation with proper cleanup"""
    
    noise_level, noise_type, config, results_dir = args
    
    # Initialize Brian2 scope per process
    start_scope()
    # set_device('cpp_standalone', build_on_run=True)  # Device optimizado
    prefs.codegen.target = 'cython' 
    
    try:
        print(f"Worker: Ejecutando noise_level = {noise_level}, type = {noise_type}")
        
        # Create simulator
        sim = IzhikevichNetwork(dt_val=config['dt_val'], T_total=config['T_total_ms'], seed_val=config['seed_val'])
        
        # Create populations with specific noise type
        pop_A = sim.create_population('A', Ne=config['Ne'], Ni=config['Ni'], 
                                    k_exc=config['k_factor']*0.5, k_inh=config['k_factor']*1.0,
                                    noise_exc=noise_level, noise_inh=noise_level*config['noise_inh_factor'], 
                                    p_intra=config['p_intra'], delay=config['intra_delay_ms'], 
                                    noise_type=noise_type)
        
        pop_B = sim.create_population('B', Ne=config['Ne'], Ni=config['Ni'], 
                                    k_exc=config['k_factor']*0.5, k_inh=config['k_factor']*1.0,
                                    noise_exc=noise_level, noise_inh=noise_level*config['noise_inh_factor'], 
                                    p_intra=config['p_intra'], delay=config['intra_delay_ms'], 
                                    noise_type=noise_type)
        
        # Inter-population connections
        syn_AB = sim.connect_populations('A', 'B', p_inter=config['p_inter'], 
                                       weight_scale=config['k_factor']*config['inter_k_factor'], 
                                       delay_value=config['delay_ms'])
        syn_BA = sim.connect_populations('B', 'A', p_inter=config['p_inter'], 
                                       weight_scale=config['k_factor']*config['inter_k_factor'], 
                                       delay_value=config['delay_ms'])
        
        # Run simulation
        sim.setup_monitors(['A', 'B'])
        results = sim.run_simulation()
        
        # Connectivity analysis
        condition_name = f"{noise_type}_{noise_level}"
        
        spike_times_A = results['A']['spike_times']
        spike_indices_A = results['A']['spike_indices']
        spike_times_B = results['B']['spike_times']
        spike_indices_B = results['B']['spike_indices']
        
        # Mock spike monitors for existing analyze_simulation_results function
        class MockSpikeMon:
            def __init__(self, times, indices):
                self.t = times * ms
                self.i = indices
        
        mock_mon_A = MockSpikeMon(spike_times_A, spike_indices_A)
        mock_mon_B = MockSpikeMon(spike_times_B, spike_indices_B)
        
        connectivity_results = analyze_simulation_results(mock_mon_A, mock_mon_B, config['N_total'], condition_name)
        
        # Generate raster plot
        plt.figure(figsize=(14, 8))
        
        # Pop A
        plt.subplot(1, 2, 1)
        exc_mask_A = spike_indices_A < config['Ne']
        inh_mask_A = spike_indices_A >= config['Ne']
        
        plt.plot(spike_times_A[exc_mask_A], spike_indices_A[exc_mask_A], '.k', markersize=0.7)
        plt.plot(spike_times_A[inh_mask_A], spike_indices_A[inh_mask_A], '.k', markersize=0.7)
        plt.axhline(y=config['Ne'], color='r', linestyle='-', linewidth=1)
        plt.xlabel('Tiempo (ms)')
        plt.ylabel('Neuronas A')
        plt.title('Population A')
        plt.ylim(0, config['N_total'])

        # Pop B
        plt.subplot(1, 2, 2)
        exc_mask_B = spike_indices_B < config['Ne']
        inh_mask_B = spike_indices_B >= config['Ne']
        
        plt.plot(spike_times_B[exc_mask_B], spike_indices_B[exc_mask_B], '.k', markersize=0.7)
        plt.plot(spike_times_B[inh_mask_B], spike_indices_B[inh_mask_B], '.k', markersize=0.7)
        plt.axhline(y=config['Ne'], color='r', linestyle='-', linewidth=1)
        plt.xlabel('Tiempo (ms)')
        plt.ylabel('Neuronas B')
        plt.title('Population B')
        plt.ylim(0, config['N_total'])
        
        plt.suptitle(f'Raster Plot - {noise_type.title()} Noise Level {noise_level}', fontsize=16)
        
        # Save raster plot
        raster_filename = f"{results_dir}/rasters/raster_{noise_type}_{noise_level}.png"
        plt.savefig(raster_filename, dpi=300, bbox_inches='tight')
        plt.close()  # Important: close figure to free memory
        
        # Network statistics
        import io
        import sys
        old_stdout = sys.stdout
        sys.stdout = buffer = io.StringIO()
        
        network_stats = print_network_statistics_table(results, sim, config['Ne'], config['Ni'], 
                                                      config['T_total_ms'], config['warmup_ms'])
        
        sys.stdout = old_stdout
        stats_output = buffer.getvalue()
        
        print(f"Worker: ✓ Completado {noise_type}_{noise_level}")
        
        # Prepare return data
        return_data = {
            'noise_level': noise_level,
            'noise_type': noise_type,
            'condition_name': condition_name,
            'connectivity_results': connectivity_results,
            'network_stats': network_stats,
            'stats_output': stats_output,
            'raster_saved': True
        }
        
    except Exception as e:
        print(f"Worker: ❌ Error en {noise_type}_{noise_level}: {str(e)}")
        return_data = {
            'noise_level': noise_level,
            'noise_type': noise_type,
            'condition_name': f"{noise_type}_{noise_level}",
            'connectivity_results': None,
            'network_stats': None,
            'stats_output': f"Error: {str(e)}",
            'raster_saved': False
        }
    
    finally:
        # ============= RESOURCE CLEANUP =============
        
        # 1. Clear Brian2 scope and devices
        try:
            from brian2 import clear_cache, device
            clear_cache('codeobj')  # Clear code object cache
            if hasattr(device, 'reinit'):
                device.reinit()
        except:
            pass
        
        # 2. Close any remaining matplotlib figures
        plt.close('all')
        
        # 3. Clear large variables from memory
        try:
            del sim, results, spike_times_A, spike_indices_A
            del spike_times_B, spike_indices_B, connectivity_results
            del mock_mon_A, mock_mon_B, analyzer
        except:
            pass
        
        # 4. Force garbage collection
        import gc
        gc.collect()
        
        # 5. Reset random seeds (optional but good practice)
        import numpy as np
        np.random.seed(None)
        
        print(f"Worker: 🧹 Recursos limpiados para {noise_type}_{noise_level}")
        
        # ============================================
    
    return return_data

In [ ]:
def run_experiment_5_parallel():
    """Experimento 5 paralelo: Efecto del delay en diferentes regímenes de actividad"""
    
    # ========== CONFIGURATION VARIABLES ==========
    # Experiment parameters
    noise_types = ['gaussian', 'poisson']
    noise_levels = [10, 18]
    delay_ms = 6
    k_factor = 20
    inter_k_factor = 5.0
    
    # Simulation parameters
    dt_val = 0.01
    T_total_ms = 1200
    warmup_ms = 200
    seed_val = 42
    
    # Population parameters
    Ne = 800
    Ni = 200
    N_total = Ne + Ni
    
    # Connectivity parameters
    p_intra = 0.1
    p_inter = 0.01
    intra_delay_ms = 0.2
    
    # Noise parameters
    noise_inh_factor = 0.45
    
    # Analysis parameters
    max_lag_ms = 120
    smooth_window = 5
    nperseg_coherence = 1024
    nperseg_psd = 512
    # =============================================
    
    # Create configuration object
    config = {
        'delay_ms': delay_ms,
        'k_factor': k_factor,
        'inter_k_factor': inter_k_factor,
        'dt_val': dt_val,
        'T_total_ms': T_total_ms,
        'warmup_ms': warmup_ms,
        'seed_val': seed_val,
        'Ne': Ne,
        'Ni': Ni,
        'N_total': N_total,
        'p_intra': p_intra,
        'p_inter': p_inter,
        'intra_delay_ms': intra_delay_ms,
        'noise_inh_factor': noise_inh_factor,
        'max_lag_ms': max_lag_ms,
        'smooth_window': smooth_window,
        'nperseg_coherence': nperseg_coherence,
        'nperseg_psd': nperseg_psd
    }
    
    # Create results directory
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    results_dir = f"./results/experiment_noise_comparison_{timestamp}"
    os.makedirs(results_dir, exist_ok=True)
    os.makedirs(results_dir+"/rasters/", exist_ok=True)
    
    print(f"=== EXPERIMENTO: GAUSSIAN vs POISSON NOISE (PARALELO) ===")
    print(f"Delay fijo: {delay_ms}ms")
    print(f"Noise types: {noise_types}")
    print(f"Noise levels: {noise_levels}")
    print(f"Total simulaciones: {len(noise_types) * len(noise_levels)}")
    print(f"Resultados guardados en: {results_dir}")
    
    # Parallel execution
    print(f"\n--- Ejecutando simulaciones en paralelo ---")
    
    # Prepare arguments for all combinations
    worker_args = []
    for noise_type in noise_types:
        for noise_level in noise_levels:
            worker_args.append((noise_level, noise_type, config, results_dir))
    
    with Pool(processes=len(worker_args)) as pool:
        worker_results = pool.map(run_single_noise_simulation, worker_args)
    
    # Merge results
    print("\n--- Procesando resultados ---")
    
    all_connectivity_results = {}
    all_network_stats = {}
    
    for result in worker_results:
        condition = result['condition_name']
        all_connectivity_results[condition] = result['connectivity_results']
        all_network_stats[condition] = result['network_stats']
        
        # Print individual stats
        print(result['stats_output'])
        
        # Move raster plots to results directory
        old_path = f"raster_noise_{result['noise_level']}.png"
        new_path = f"{results_dir}/raster_noise_{result['noise_level']}.png"
        try:
            import shutil
            shutil.move(old_path, new_path)
        except:
            print(f"Warning: Could not move {old_path}")
    
    # Generate connectivity dashboard
    print("\n--- Generando dashboard de conectividad ---")
    
    fig1 = plot_connectivity_dashboard(all_connectivity_results, figsize=(18, 12))
    fig1.suptitle('Experiment 5: Connectivity Analysis Across Noise Levels', fontsize=16)
    fig1.savefig(f"{results_dir}/connectivity_dashboard.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    # Create summary table
    summary_data = create_summary_table(all_connectivity_results, all_network_stats)
    summary_df = pd.DataFrame(summary_data)
    summary_df.to_csv(f"{results_dir}/summary_metrics.csv", index=False)
    
    # Generate trends plot
    plot_summary_trends(summary_df, results_dir)
    
    # Save comprehensive configuration
    final_config = {
        'experiment': 'Régimen de Actividad (Paralelo)',
        'timestamp': timestamp,
        'parallel': True,
        'noise_levels': noise_levels,
        
        # All configuration variables automatically included
        **config
    }
    
    with open(f"{results_dir}/experiment_config.json", 'w') as f:
        json.dump(final_config, f, indent=2)
    
    print(f"\n✓ Experimento paralelo completado!")
    print(f"📁 Resultados guardados en: {results_dir}")
    print(f"📊 Archivos generados:")
    print(f"  - connectivity_dashboard.png")
    print(f"  - summary_metrics.csv")
    print(f"  - trends_summary.png")
    print(f"  - experiment_config.json")
    print(f"  - raster_noise_*.png")
    
    # Print comprehensive summary
    print_experiment_summary(all_connectivity_results, summary_df, results_dir, delay_ms, noise_levels)
    
    return all_connectivity_results, summary_df, results_dir

In [ ]:
# Ejecutar experimento paralelo
connectivity_results, summary_df, results_dir = run_experiment_5_parallel()

# Mostrar resumen final
print("\n=== RESUMEN CUANTITATIVO ===")
print(summary_df[['noise_level', 'cross_corr_peak', 'plv_alpha', 'firing_rate_exc_A', 'cv_A']].round(3))

In [ ]:
def validate_analysis_results(results_dict, summary_df):
    """Comprehensive validation of analysis results"""
    
    print("=== VALIDATION REPORT ===\n")
    
    # 1. Cross-correlation lag validation
    print("1. CROSS-CORRELATION LAGS:")
    for _, row in summary_df.iterrows():
        lag = row['cross_corr_lag_ms']
        condition = row['condition']
        expected_delay = 6  # ms
        
        if abs(lag) > 20:
            print(f"  ❌ {condition}: lag={lag:.1f}ms (too high)")
        elif lag < 0:
            print(f"  ⚠️  {condition}: lag={lag:.1f}ms (negative - check causality)")
        elif abs(lag - expected_delay) < 2:
            print(f"  ✅ {condition}: lag={lag:.1f}ms (close to expected)")
        else:
            print(f"  ⚠️  {condition}: lag={lag:.1f}ms (deviation from expected)")
    
    # 2. PLV/PLI consistency validation
    print("\n2. PLV/PLI CONSISTENCY:")
    for _, row in summary_df.iterrows():
        plv = row['plv_alpha']
        pli = row['pli_alpha']
        condition = row['condition']
        
        if pli > plv + 0.1:  # PLI shouldn't exceed PLV significantly
            print(f"  ❌ {condition}: PLI({pli:.3f}) > PLV({plv:.3f}) - impossible")
        elif abs(plv - pli) > 0.5:  # Large differences suspicious
            print(f"  ⚠️  {condition}: PLV({plv:.3f}) vs PLI({pli:.3f}) - large diff")
        else:
            print(f"  ✅ {condition}: PLV({plv:.3f}), PLI({pli:.3f}) - consistent")
    
    # 3. Firing rate vs CV relationship
    print("\n3. FIRING RATE vs CV RELATIONSHIP:")
    for _, row in summary_df.iterrows():
        firing_rate = row['firing_rate_exc_A']
        cv = row['cv_A']
        noise_level = row['noise_level']
        noise_type = row['noise_type']
        
        print(f"  {noise_type}_{noise_level}: Rate={firing_rate:.1f}Hz, CV={cv:.3f}")
    
    # Check if CV decreases with noise (suspicious)
    gaussian_data = summary_df[summary_df['noise_type'] == 'gaussian'].sort_values('noise_level')
    poisson_data = summary_df[summary_df['noise_type'] == 'poisson'].sort_values('noise_level')
    
    print("\n4. CV TREND VALIDATION:")
    for data, name in [(gaussian_data, 'Gaussian'), (poisson_data, 'Poisson')]:
        cv_values = data['cv_A'].values
        if len(cv_values) > 1:
            trend = "decreasing" if cv_values[-1] < cv_values[0] else "increasing"
            print(f"  {name}: CV trend is {trend} with noise level")
            if trend == "decreasing":
                print(f"    ⚠️  This is counterintuitive - needs investigation")
    
    # 5. Coherence peaks validation
    print("\n5. COHERENCE PEAK VALIDATION:")
    peak_freqs = summary_df['coherence_freq_hz'].values
    unique_peaks = np.unique(peak_freqs)
    
    print(f"  Unique peak frequencies: {unique_peaks}")
    if len(unique_peaks) < 3:
        print(f"  ⚠️  Too few unique peaks - possible discretization artifact")
    
    # Count zeros and repeated values
    zero_count = np.sum(peak_freqs == 0.0)
    repeated_13_count = np.sum(peak_freqs == 13.333333333333334)
    
    print(f"  Peaks at 0.0Hz: {zero_count}/{len(peak_freqs)}")
    print(f"  Peaks at 13.3Hz: {repeated_13_count}/{len(peak_freqs)}")
    
    if zero_count > len(peak_freqs)//2:
        print(f"    ❌ Too many zero-frequency peaks - check spectral analysis")
    
    # 6. Activity regime validation
    print("\n6. ACTIVITY REGIME VALIDATION:")
    def correct_classify_regime(firing_rate):
        if firing_rate < 0.5:
            return "Silent"
        elif firing_rate < 2.0:
            return "Low Active"  
        elif firing_rate < 15.0:
            return "Active"
        else:
            return "Bursting"
    
    for _, row in summary_df.iterrows():
        firing_rate = row['firing_rate_exc_A']
        reported_regime = row['sync_level_A']  # This seems wrong - should be activity level
        correct_regime = correct_classify_regime(firing_rate)
        condition = row['condition']
        
        print(f"  {condition}: Rate={firing_rate:.1f}Hz → Should be '{correct_regime}'")
    
    # 7. Timescale validation
    print("\n7. INTRINSIC TIMESCALE VALIDATION:")
    timescales_A = summary_df['timescale_A_ms'].values
    timescales_B = summary_df['timescale_B_ms'].values
    
    print(f"  Timescale A range: {np.min(timescales_A):.1f} - {np.max(timescales_A):.1f} ms")
    print(f"  Timescale B range: {np.min(timescales_B):.1f} - {np.max(timescales_B):.1f} ms")
    
    # Check if all are very similar (suspicious)
    if (np.max(timescales_A) - np.min(timescales_A)) < 10:
        print(f"    ⚠️  Very narrow range for timescales A - check calculation")
    
    # 8. Summary recommendations
    print("\n=== RECOMMENDATIONS ===")
    print("1. Recalculate cross-correlation with proper lag normalization")
    print("2. Verify PLV/PLI calculation - use different frequency bands")
    print("3. Check CV calculation - should increase with noise level")
    print("4. Fix activity regime classification logic")
    print("5. Investigate coherence discretization artifacts")
    print("6. Validate spectral analysis parameters (nperseg, fs)")
    
    return True

# Additional function to check raw data integrity
def check_spike_data_integrity(results):
    """Check if spike data makes sense"""
    print("\n=== SPIKE DATA INTEGRITY ===")
    
    for pop_name in ['A', 'B']:
        spike_times = results[pop_name]['spike_times']
        spike_indices = results[pop_name]['spike_indices']
        
        print(f"\nPopulation {pop_name}:")
        print(f"  Total spikes: {len(spike_times)}")
        print(f"  Time range: {np.min(spike_times):.1f} - {np.max(spike_times):.1f} ms")
        print(f"  Neuron range: {np.min(spike_indices)} - {np.max(spike_indices)}")
        
        # Check for anomalies
        if len(spike_times) == 0:
            print(f"    ❌ No spikes detected!")
        elif np.max(spike_indices) >= 1000:
            print(f"    ❌ Neuron indices exceed population size!")
        elif np.min(spike_times) < 0:
            print(f"    ❌ Negative spike times!")
        else:
            print(f"    ✅ Spike data looks valid")

# Usage in your experiment:
validate_analysis_results(connectivity_results, summary_df)


In [ ]:
check_spike_data_integrity(connectivity_results['gaussian_10'])  # For one set of results